In [ ]:
# 导入库
import akshare as ak
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# akshare 版本  
print(ak.__version__)

In [ ]:
# 获取白酒行业成分股数据

stocks = ak.stock_board_industry_cons_ths(symbol="白酒")
stocks.to_csv("白酒股票.csv")


In [ ]:
# 获取板块股票逐年利润表
profit_y_dict = dict()

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    if (code.startswith('00') != True):
        market ='sh'
    symbol = "{}{}".format(market, code)
    profit_y = ak.stock_profit_sheet_by_yearly_em(symbol=symbol)

    profit_y_dict[code] = profit_y

In [ ]:
# 获取板块股票归属母公司利润表

profit_dict = {}
for i in tqdm(range(len(stocks))):
    code = stocks.loc[i, "代码"]

    p = pd.Series(list(profit_y_dict[code]["PARENT_NETPROFIT"]), index=pd.to_datetime(profit_y_dict[code]["REPORT_DATE"]).dt.date)
    profit_dict[code] = p

df_parent_profit = pd.DataFrame(profit_dict)

In [ ]:
df_parent_profit['000858']

In [ ]:
df_parent_profit['000858'].to_csv("profit_000858.csv")

In [ ]:
df_eps = pd.DataFrame()
eps_dict = {}

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]
    
    s1 = pd.Series(list(profit_y_dict[code]["BASIC_EPS"]), index=pd.to_datetime(profit_y_dict[code]["REPORT_DATE"]))
    eps_dict["{}-{}".format(code, name)] = s1

df_eps = pd.DataFrame(eps_dict)
df_eps.fillna(0, inplace=True)

In [ ]:
df_eps.to_csv("eps.csv")

In [ ]:
df_eps.fillna(0, inplace=True)


In [ ]:
print(type(df_eps.index))

In [ ]:
df_eps['2010':]

In [ ]:
df_eps['2010':].plot(kind='bar')

In [ ]:
print(df_eps.mean().sort_values(ascending=False))

In [ ]:
df_eps.sort_index(ascending=True).plot(kind="bar", colormap="cubehelix",legend=False)

In [ ]:
### 导出每股收益到csv
profiet_y = ak.stock_profit_sheet_by_yearly_em(symbol="sh603919")
print(profit_y["BASIC_EPS"][0:5])
profiet_y.to_csv("profit_603919.csv")

In [ ]:
df_stocks.sort_index(ascending=True).pct_change().plot(kind="bar", colormap="cubehelix")

In [ ]:
#获取交易日期

import datetime

trade_date_list= ak.tool_trade_date_hist_sina()
all_trade_date = pd.to_datetime(trade_date_list["trade_date"])
all_trade_date.index = all_trade_date
print(all_trade_date)

ts = pd.Series(np.random.randn(len(trade_date_index)), index=trade_date_index)

enddate = datetime.datetime.now().strftime("%Y%m%d")
print(ts)
print(ts['2016-01-01':enddate])

In [ ]:
# 导出白酒成分股到csv
stocks.to_csv("bj.csv")

In [ ]:
#股票股价走势
import datetime

startdate = datetime.date(2000,1,1)
enddate = datetime.datetime.now().strftime("%Y%m%d")

trade_date_list= ak.tool_trade_date_hist_sina()
all_trade_date = pd.to_datetime(trade_date_list["trade_date"])
all_trade_date.index = all_trade_date
trade_date_index = all_trade_date[startdate:enddate]

cl_dict = {}

for i in tqdm(range(len(stocks))):
# for i in range(3):
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]
    stock_hist = ak.stock_zh_a_hist(symbol=code, period="daily", start_date=startdate.strftime("%Y%m%d"), end_date=enddate, adjust="hfq")
    
    stock_hist.to_csv(f"{code}-stock.csv")

    cl = stock_hist["收盘"]

    ts = pd.Series(list(cl), index=pd.to_datetime(stock_hist["日期"]).dt.date)
    ts_high = pd.Series(list(stock_hist["最高"]), index=pd.to_datetime(stock_hist["日期"]).dt.date)
    ts_low = pd.Series(list(stock_hist["最低"]), index=pd.to_datetime(stock_hist["日期"]).dt.date)

    cl_dict["{}".format(code)] = ts
df_close = pd.DataFrame(cl_dict)  

In [ ]:
stocks

In [ ]:
ret = (1 + df_close.pct_change()).cumprod()
print(ret)
# ret.to_csv("ret.csv")
# df_close.to_csv("cl.csv")

# print(df_close.loc["2016-01-29",  :])
# print(df_close.resample('M').asfreq())

In [ ]:
ret.plot(kind="line")

In [ ]:
print(df_close['600519'].resample('M').last())

In [ ]:
print(df_close)

In [ ]:
### 月收益率
month_ret = df_close.pct_change().resample('M').apply(lambda x : ((1+x).prod() - 1))
print(month_ret)

In [ ]:
### 年度收益率
year_ret = df_close.pct_change().resample('Y').apply(lambda x : ((1+x).prod() - 1))
print(year_ret)

In [ ]:
### 作图区

ax1 = df_parent_profit['000858'].dropna().sort_index().plot(kind="bar")

pc = df_parent_profit['000858'].dropna().sort_index().pct_change()
pc.fillna(0, inplace=True)

ax2 = ax1.twinx()
ax2.set_ylabel('change rate')
ax2.set_ylim(-1,1)
pc.plot(kind='line', marker='*', color='red', ms=10)

In [ ]:
year_ret['000858'].plot(kind='bar')

In [ ]:
df_dividents = pd.DataFrame()
dividents_dict = {}

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]

    dividents = ak.stock_dividents_cninfo(symbol=code)
    # sg = pd.Series(list(dividents["送股比例"]), index=pd.to_datetime(dividents["日期"]))
    # zz = pd.Series(list(dividents["转增比例"]), index=pd.to_datetime(dividents["日期"]))
    dividents.to_csv(f"{code}-div.csv")
    px = pd.Series(list(dividents["派息比例"]), index=pd.to_datetime(dividents["报告时间"]))

    dividents_dict["{}-{}".format(code, name)] = px

df_dividents = pd.DataFrame(dividents_dict)
df_dividents.fillna(0, inplace=True)

In [ ]:
import akshare as ak
from tqdm import tqdm 
import pandas as pd


def balance_sheet_analyze(industry : str, name: str):
    stocks = ak.stock_board_industry_cons_ths(symbol=industry)
    index = ['资产总计', '负债合计', '所有者权益(或股东权益)合计', '未分配利润']
    columns =  []
    # date_index = None
    report_all = pd.DataFrame()

    for i in range(len(stocks)):
        code = stocks.loc[i, "代码"]
        columns.append(code)

    for i in tqdm(range(len(stocks))):
        code = stocks.loc[i, "代码"]

        balance = ak.stock_financial_report_sina(stock=code, symbol=name)
        # if date_index is None:
            # print(pd.to_datetime(balance["报表日期"]))
        date_index = pd.to_datetime(balance["报表日期"])

        # if report_all is None:
        #     report_all = pd.DataFrame(
        #         # np.random.randn(204,1),
        #         columns=columns,
        #         index=pd.MultiIndex.from_product(
        #             [date_index, index]
        #         )
        #     )            
        report = pd.DataFrame(
            # np.random.randn(204,1),
            columns=[code],
            index=pd.MultiIndex.from_product(
                [date_index, index]
            )
        )   #             

        for idx in range(len(index)):
            # tmpDf = pd.DataFrame(balance[index[idx]], index=pd.to_datetime(balance["报表日期"]))
            report.loc[(slice(None), index[idx]), :] = balance[index[idx]].values
        # print(report_all)
        report_all = pd.concat([report_all, report], axis=1)
        # print(report_all)

    return report_all

In [ ]:
df = balance_sheet_analyze("白酒", "资产负债表")

df.to_csv("bj_bal.csv", encoding="utf_8_sig")
# print(df)

In [ ]:
df.loc['2022-09', '000568']

In [ ]:
df.sort_index(level=0).loc['2018-01':'2022-12', '资产总计', :].fillna(0).applymap(lambda x:format(float(x),','))

In [ ]:
df.sort_index(level=0).loc['2018-01':'2022-12', '资产总计', :]

In [ ]:
df.sort_index(level=0).loc['2018-01':'2022-12', '资产总计'].applymap(lambda x:format(int(x),','))

idx = pd.IndexSlice
df.sort_index(level=0).loc["2018-01":"2022-09", :].apply(lambda x: format(x, ','))